In [ ]:
import os
import numpy as np
import polars as pl
from torch.utils.data import Dataset
from torchvision.io import decode_image
import matplotlib.pyplot as plt

In [ ]:
pl.Config.set_tbl_rows(-1)
pl.Config.set_tbl_width_chars(-1)
pl.Config.set_tbl_cols(-1)
pl.Config.set_fmt_table_cell_list_len(-1)
pl.Config.set_fmt_str_lengths(1000)

In [ ]:
root_dir = os.path.expanduser('/scr/data/cell_crops/')
metadata_path = os.path.join(root_dir, 'metadata.csv')

In [ ]:
metadata = pl.read_csv(metadata_path)
first_entry = metadata[0]
first_entry

In [ ]:
metadata.__len__()

In [ ]:
dict_meta = metadata.rows(named=True)
print(len(dict_meta))
dict_meta[0]

In [ ]:
first_entry

In [ ]:
studies = open("test_antibodies.txt", 'r').readlines()
studies = [s.strip() for s in studies if s.strip()]

In [ ]:
filtered_antibodies = metadata.filter(pl.col('antibody').is_in(studies))


In [ ]:
plate = str(first_entry['if_plate_id'].item())
position = str(first_entry['position'].item())
sample = str(first_entry['sample'].item())
cell_id = sample = str(int(first_entry['cell_id'].item()))
image = os.path.join(root_dir, plate, f"{plate}_{position}_{sample}_{cell_id}_cell_image.png")
pask = os.path.join(root_dir, plate, f"{plate}_{position}_{sample}_{cell_id}_cell_mask.png")

In [ ]:
metadata.head()

In [ ]:
import os
import numpy as np
import polars as pl
from torch.utils.data import Dataset
from torchvision.io import decode_image
import matplotlib.pyplot as plt
import torch

class UnZippedImageArchive(Dataset):
    """Basic unzipped image arch. This will no longer be used. 
       Remove when unzipped support is added to the IterableImageArchive
    """
    def __init__(self, root_dir: str= '/scr/data/cell_crops/') -> None:
        super().__init__()
        self.root_dir = root_dir
        self.metadata_path = os.path.join(self.root_dir, 'metadata.csv')
        self.metadata = pl.read_csv(self.metadata_path).rows(named=True)
        
    def __len__(self):
        return len(self.metadata)
    
    def __getitem__(self, idx):
        # microtubule fluorescence,  Blue (B) channel
        # endoplasmic reticulum,  Green (G) channel
        # DNA, Red (R) channel
        # Protein of interest, Alpha (A) channel
        # https://virtualcellmodels.cziscience.com/dataset/01933229-3c87-7818-be80-d7e5578bb0b7
        row = self.metadata[idx]
        plate = str(row['if_plate_id'])
        position = row['position']
        sample = str(row['sample'])
        cell_id = str(int(row['cell_id']))
        image = os.path.join(self.root_dir, plate, f"{plate}_{position}_{sample}_{cell_id}_cell_image.png")
        image = decode_image(image, mode='RGBA'), decode_image(mask)
        
        # bool_mask = (mask>0).expand((image.shape[0], -1, -1))
        # return torch.where(bool_mask, image, 0)
        return image
        
dataset = UnZippedImageArchive()

In [12]:
dataset = UnZippedImageArchive()
image = dataset[5] # 4, 1024, 1024

channels = [channel.unsqueeze(0) for channel in image]
concatenated_image = torch.cat(channels[:-1], dim=0)
image_data_rgb = np.transpose(concatenated_image, (1, 2, 0))

image_data_rgb.shape
plt.imshow(image_data_rgb)
plt.axis('off')
plt.savefig(f'full_slide_hpa.png', bbox_inches='tight', pad_inches=0)

# for ch in range(concatenated_image.shape[0]):
#     channel = concatenated_image[ch, : , :]
#     plt.imshow(channel, cmap='grey')
#     plt.axis('off')
#     plt.savefig(f'ch_{ch}.png', bbox_inches='tight', pad_inches=0)

AttributeError: 'str' object has no attribute 'value'

In [5]:
import numpy as np

# Load the file
load_features = np.load('/scr/data/HPA_features/all_features.pth')

# See all available keys
print("Available keys:", list(load_features.keys()))

# Access specific data using the keys
data_pkl = load_features['all_features/data.pkl']
data_0 = load_features['all_features/data/0']
version = load_features['all_features/version']

# Print shapes and types to understand the data
print(f"data.pkl shape: {data_pkl.shape if hasattr(data_pkl, 'shape') else 'No shape'}")
print(f"data/0 shape: {data_0.shape if hasattr(data_0, 'shape') else 'No shape'}")
print(f"version: {version}")

# Don't forget to close the file when done
load_features.close()

Available keys: ['all_features/data.pkl', 'all_features/data/0', 'all_features/version']
data.pkl shape: No shape
data/0 shape: No shape
version: b'3\n'


In [ ]:
import torch
features = torch.load('/scr/data/HPA_features/all_features.pth')

In [23]:
import pandas as pd


metadata = pd.DataFrame(features[0])
metadata.shape

(1138686, 14)

In [14]:
features[1].size()

torch.Size([1138688, 1536])